In [25]:
!pip install huggingface-hub
!pip install -q flash-attn --no-build-isolation
!pip install  -U -q transformers trl datasets bitsandbytes peft accelerate
!pip install qwen_vl_utils

Found existing installation: unsloth 2025.1.7
Uninstalling unsloth-2025.1.7:
  Successfully uninstalled unsloth-2025.1.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-deg0bzhi
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-deg0bzhi

  Resolved https://github.com/unslothai/unsloth.git to commit bdf0cd6033595be4e7ed23d0d002bb176d343152
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.7-py3-none-any.whl size=174896 sha256=65ae16a295fb0b62f106742e47ce61af7d27481982ee3fe5eafc9ad638bd6a55
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8hr6peg/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A ne

In [4]:
from datasets import load_dataset

ds = load_dataset("krowiemlekommm/PJN_CHARTS")

In [ ]:
from huggingface_hub import notebook_login
import torch
notebook_login()

In [5]:
ds['train']

Dataset({
    features: ['image', 'query', 'label'],
    num_rows: 1200
})

In [9]:
message = """You are a Vision Language Model for analyzing chart images. 
Provide concise answers (e.g., a word, number, or short phrase) based on the chart's visual data. 
Focus on accuracy and avoid unnecessary explanations."""

def format_data(sample):
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": message
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"],
                },
                {
                    "type": "text",
                    "text": sample['query'],
                }
            ],
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": sample["label"][0]
                }
            ],
        },
    ]

In [10]:
train_dataset = [format_data(sample) for sample in ds['train']]
eval_dataset = [format_data(sample) for sample in ds['val']]
test_dataset = [format_data(sample) for sample in ds['test']]

In [11]:
import gc
import time

def clear_memory():

    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'peft_model' in globals(): del globals()['peft_model']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)


    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

In [16]:
from transformers import BitsAndBytesConfig
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
import torch

model_id = "Qwen/Qwen2-VL-2B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [17]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)


peft_model = get_peft_model(model, peft_config)


peft_model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 2,210,075,136 || trainable%: 0.0493


In [18]:
from trl import SFTConfig

training_args = SFTConfig(
    output_dir="smolvlm",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=1,
    optim="adamw_torch_fused",
    bf16=True,
    push_to_hub=True,
    report_to="wandb",
    remove_unused_columns=False,
    gradient_checkpointing=True,
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

In [19]:
import wandb
wandb.login(key="")
wandb.init(
    project="PJN_qwen_2b",
    name="PJN_qwen_2b",  
    config=training_args,
)

wandb: Currently logged in as: s26048 (s26048-polish-japanese-academy-of-information-technology-). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [26]:
from qwen_vl_utils import process_vision_info


def collate_fn(examples):

    texts = [processor.apply_chat_template(example, tokenize=False) for example in examples]  
    image_inputs = [process_vision_info(example)[0] for example in examples]  

    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True) 
    labels = batch["input_ids"].clone()  
    labels[labels == processor.tokenizer.pad_token_id] = -100  

    
    if isinstance(processor, Qwen2VLProcessor):  
        image_tokens = [151652, 151653, 151655]  
    else:
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]


    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100  

    batch["labels"] = labels  

    return batch

In [27]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

/tmp/ipykernel_1679/93380869.py:3: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [28]:
trainer.train()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
25,3.691400
50,2.908000
75,1.586300
100,0.889700
125,0.794900
150,0.748900


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_ree

TrainOutput(global_step=150, training_loss=1.769874356587728, metrics={'train_runtime': 1011.161, 'train_samples_per_second': 1.187, 'train_steps_per_second': 0.148, 'total_flos': 9563123430604800.0, 'train_loss': 1.769874356587728, 'epoch': 1.0})

In [30]:
model.push_to_hub("krowiemlekommm/PJN_qwen_2b", token = "")
processor.push_to_hub("krowiemlekommm/PJN_qwen_2b", token = "")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/krowiemlekommm/PJN_qwen_2b/commit/ef4ed4c3cf83b2ce6febd72a34896da96db4d2e4', commit_message='Upload processor', commit_description='', oid='ef4ed4c3cf83b2ce6febd72a34896da96db4d2e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/krowiemlekommm/PJN_qwen_2b', endpoint='https://huggingface.co', repo_type='model', repo_id='krowiemlekommm/PJN_qwen_2b'), pr_revision=None, pr_num=None)

In [31]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▇▅▂▁▁
train/learning_rate,▅█▆▅▃▁
train/loss,█▆▃▁▁▁
total_flos,9563123430604800.0
train/epoch,1
train/global_step,150
train/grad_norm,0.63376
train/learning_rate,0
train/loss,0.7489
